In [ ]:
"""
================================================================================
KAGGLE-OPTIMIZED MULTI-CLASS CLASSIFICATION PIPELINE
================================================================================
A fast, competition-ready ML pipeline optimized for Kaggle environments:
- Uses pre-installed libraries only (no pip install needed)
- Executes in 2-3 minutes
- Automatic format matching with sample submission
- Production-grade accuracy with minimal configuration
================================================================================
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
                              ExtraTreesClassifier, VotingClassifier)
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import (roc_auc_score, log_loss, classification_report,
                            accuracy_score, f1_score)
from sklearn.impute import SimpleImputer

# Import XGBoost (pre-installed on Kaggle)
try:
    from xgboost import XGBClassifier
    XGBOOST_AVAILABLE = True
except ImportError:
    XGBOOST_AVAILABLE = False

# Import LightGBM (pre-installed on Kaggle)
try:
    from lightgbm import LGBMClassifier
    LIGHTGBM_AVAILABLE = True
except ImportError:
    LIGHTGBM_AVAILABLE = False

import warnings
warnings.filterwarnings("ignore")

import time
start_time = time.time()


# ============================================================================
# 1. USER CONFIGURATION - MODIFY THIS SECTION ONLY
# ============================================================================

# ============================================================================
# 1. USER CONFIGURATION - MODIFY THIS SECTION ONLY
# ============================================================================

CONFIG = {
    # File paths (Kaggle default paths)
    'train_path': "/kaggle/input/mle-ese-mock/train (5).csv",
    'test_path': "/kaggle/input/mle-ese-mock/test (4).csv",
    'sample_submission_path': "/kaggle/input/mle-ese-mock/sample_submission.csv",
    'output_file': "submission.csv",

    # Column configuration
    'target_col': "quality_grade",  # Can be string or list for multi-target
    'id_col': "id",

    # Competition type
    'submission_type': 'probabilities',  # 'probabilities' or 'classes'
    'submission_column_prefix': 'Status_',  # Prefix for output columns (e.g., 'Status_')

    # Model configuration (optimized for speed and accuracy)
    'model_type': 'lightgbm',  # Options: 'lightgbm', 'xgboost', 'random_forest', 'fast_ensemble'
    'n_estimators': 500,  # Reduced for speed (500 is sweet spot for 2-3 min)
    'use_class_weight': True,

    # Fast ensemble (combines top 2 models quickly)
    'fast_ensemble_models': ['lightgbm', 'xgboost'],  # Only 2-3 models for speed

    # Processing options (optimized for speed)
    'do_plotting': False,  # Disable plots to save time
    'do_outlier_cap': False,  # Skip outlier capping for speed
    'do_hyperparam_tuning': False,  # Disable tuning for speed (use optimized defaults)

    # Advanced options
    'validation_size': 0.2,
    'random_state': 42,
    'verbose': True,  # Show progress
}

RANDOM_STATE = CONFIG['random_state']


# ============================================================================
# 2. HELPER FUNCTIONS
# ============================================================================

def print_section(title):
    """Print a formatted section header"""
    if CONFIG.get('verbose', True):
        print("\n" + "="*80)
        print(title.upper())
        print("="*80)

def print_subsection(title):
    """Print a formatted subsection header"""
    if CONFIG.get('verbose', True):
        print("\n" + "-"*80)
        print(title)
        print("-"*80)

def detect_feature_types(df, exclude_cols=None):
    """Automatically detect categorical and numerical columns"""
    if exclude_cols is None:
        exclude_cols = []

    df = df.drop(columns=exclude_cols, errors='ignore')

    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    num_cols = df.select_dtypes(include=['number']).columns.tolist()

    # Additional check: if numerical column has few unique values, treat as categorical
    for col in num_cols.copy():
        if df[col].nunique() < 10 and df[col].nunique() < len(df) * 0.05:
            print(f"  ℹ '{col}' has few unique values ({df[col].nunique()}), treating as categorical")
            cat_cols.append(col)
            num_cols.remove(col)

    return cat_cols, num_cols

def safe_roc_auc(y_true, y_pred_proba, multi_class='ovr'):
    """Safely compute ROC AUC with error handling"""
    try:
        if len(np.unique(y_true)) == 2:
            return roc_auc_score(y_true, y_pred_proba[:, 1])
        else:
            return roc_auc_score(y_true, y_pred_proba,
                               multi_class=multi_class, average='macro')
    except Exception as e:
        print(f"  ⚠ Could not compute ROC AUC: {e}")
        return None

def get_model(model_type, n_estimators, use_class_weight, random_state):
    """
    Get optimized model with speed-accuracy balance
    """
    class_weight = 'balanced' if use_class_weight else None

    if model_type == 'lightgbm':
        if not LIGHTGBM_AVAILABLE:
            print("⚠ LightGBM not available, using Random Forest")
            return get_model('random_forest', n_estimators, use_class_weight, random_state)
        return LGBMClassifier(
            n_estimators=n_estimators,
            random_state=random_state,
            learning_rate=0.05,
            max_depth=7,
            num_leaves=50,
            subsample=0.8,
            colsample_bytree=0.8,
            n_jobs=-1,
            verbose=-1,
            class_weight=class_weight,
            min_child_samples=20
        )

    elif model_type == 'xgboost':
        if not XGBOOST_AVAILABLE:
            print("⚠ XGBoost not available, using Random Forest")
            return get_model('random_forest', n_estimators, use_class_weight, random_state)
        return XGBClassifier(
            n_estimators=n_estimators,
            random_state=random_state,
            learning_rate=0.05,
            max_depth=6,
            min_child_weight=1,
            subsample=0.8,
            colsample_bytree=0.8,
            n_jobs=-1,
            eval_metric='logloss',
            tree_method='hist',  # Faster training
            verbosity=0
        )

    elif model_type == 'random_forest':
        return RandomForestClassifier(
            n_estimators=n_estimators,
            random_state=random_state,
            class_weight=class_weight,
            n_jobs=-1,
            max_depth=15,
            min_samples_split=10,
            min_samples_leaf=4,
            max_features='sqrt'
        )

    elif model_type == 'extra_trees':
        return ExtraTreesClassifier(
            n_estimators=n_estimators,
            random_state=random_state,
            class_weight=class_weight,
            n_jobs=-1,
            max_depth=15,
            min_samples_split=10
        )

    elif model_type == 'gradient_boosting':
        return GradientBoostingClassifier(
            n_estimators=min(n_estimators, 300),  # GB is slower, limit trees
            random_state=random_state,
            max_depth=5,
            learning_rate=0.1,
            subsample=0.8
        )

    else:
        raise ValueError(f"Unknown model type: {model_type}")


def get_fast_ensemble(ensemble_models, n_estimators, use_class_weight, random_state):
    """
    Create fast voting ensemble (2-3 models max for speed)
    """
    print(f"\n✓ Building Fast Ensemble: {ensemble_models}")

    estimators = []
    for model_name in ensemble_models[:3]:  # Limit to 3 models max
        model = get_model(model_name, n_estimators, use_class_weight, random_state)
        estimators.append((model_name, model))
        print(f"  + {model_name}")

    return VotingClassifier(
        estimators=estimators,
        voting='soft',
        n_jobs=-1
    )
    """Plot distribution of target variable(s)"""
    n_targets = len(target_cols) if isinstance(target_cols, list) else 1
    target_cols = [target_cols] if isinstance(target_cols, str) else target_cols

    fig, axes = plt.subplots(1, n_targets, figsize=(figsize_per_plot*n_targets, 5))
    if n_targets == 1:
        axes = [axes]

    for idx, target_col in enumerate(target_cols):
        counts = data[target_col].value_counts().sort_index()
        axes[idx].bar(range(len(counts)), counts.values)
        axes[idx].set_title(f"Distribution: '{target_col}'")
        axes[idx].set_xlabel("Class")
        axes[idx].set_ylabel("Count")
        axes[idx].set_xticks(range(len(counts)))
        axes[idx].set_xticklabels(counts.index, rotation=45, ha='right')

        # Add count labels on bars
        for i, v in enumerate(counts.values):
            axes[idx].text(i, v, str(v), ha='center', va='bottom', fontsize=8)

    plt.tight_layout()
    plt.show()


# ============================================================================
# 3. DATA LOADING AND INITIAL INSPECTION
# ============================================================================

# ============================================================================
# 3. DATA LOADING AND INITIAL INSPECTION
# ============================================================================

print_section("DATA LOADING")

# Load data
train_data = pd.read_csv(CONFIG['train_path'])
test_data = pd.read_csv(CONFIG['test_path'])

print(f"\n✓ Data loaded: Train {train_data.shape}, Test {test_data.shape}")

# Load sample submission
sample_submission = None
if CONFIG.get('sample_submission_path'):
    try:
        sample_submission = pd.read_csv(CONFIG['sample_submission_path'])
        print(f"✓ Sample submission: {sample_submission.shape}")
        if CONFIG.get('verbose'):
            print(f"  Columns: {list(sample_submission.columns)}")
    except Exception as e:
        print(f"⚠ Could not load sample submission: {e}")

# Save test IDs
test_ids = test_data[CONFIG['id_col']].copy() if CONFIG['id_col'] in test_data.columns else None

# Drop ID columns
if CONFIG['id_col'] in train_data.columns:
    train_data = train_data.drop(columns=[CONFIG['id_col']])
if CONFIG['id_col'] in test_data.columns:
    test_data = test_data.drop(columns=[CONFIG['id_col']])


# ============================================================================
# 4. TARGET CONFIGURATION
# ============================================================================

print_section("TARGET CONFIGURATION")

# Normalize target_col to list
if isinstance(CONFIG['target_col'], str):
    target_cols = [CONFIG['target_col']]
    is_multi_target = False
    print(f"\n✓ Single target: '{CONFIG['target_col']}'")
elif isinstance(CONFIG['target_col'], list):
    target_cols = CONFIG['target_col']
    is_multi_target = True
    print(f"\n✓ Multi-target: {len(target_cols)} targets")
else:
    raise ValueError("TARGET_COL must be string or list")

# Verify targets exist
missing_targets = [col for col in target_cols if col not in train_data.columns]
if missing_targets:
    raise ValueError(f"Target columns not found: {missing_targets}")

# Analyze targets
target_info = {}
for target_col in target_cols:
    unique_classes = sorted(train_data[target_col].dropna().unique())
    n_classes = len(unique_classes)

    if CONFIG.get('verbose'):
        print(f"\n{target_col}: {n_classes} classes")
        value_counts = train_data[target_col].value_counts()
        min_pct = (value_counts.min() / len(train_data)) * 100
        if min_pct < 5:
            print(f"  ⚠ Imbalanced! Min class: {min_pct:.1f}%")

    target_info[target_col] = {
        'n_classes': n_classes,
        'classes': unique_classes
    }


# ============================================================================
# 5. DATA CLEANING
# ============================================================================

print_section("DATA CLEANING")

# Remove duplicates
train_dups = train_data.duplicated().sum()
if train_dups > 0:
    print(f"✓ Removing {train_dups} duplicate rows")
    train_data = train_data.drop_duplicates().reset_index(drop=True)

# Missing values analysis
print_subsection("Missing Values Analysis")

train_missing = train_data.isnull().sum()
test_missing = test_data.isnull().sum()

print("\nTrain missing values:")
if train_missing.sum() == 0:
    print("  ✓ No missing values")
else:
    missing_cols = train_missing[train_missing > 0]
    for col, count in missing_cols.items():
        pct = (count / len(train_data)) * 100
        print(f"  {col}: {count} ({pct:.2f}%)")

print("\nTest missing values:")
if test_missing.sum() == 0:
    print("  ✓ No missing values")
else:
    missing_cols = test_missing[test_missing > 0]
    for col, count in missing_cols.items():
        pct = (count / len(test_data)) * 100
        print(f"  {col}: {count} ({pct:.2f}%)")


# ============================================================================
# 6. FEATURE EXTRACTION AND SEPARATION
# ============================================================================

print_section("FEATURE EXTRACTION")

# Separate features and targets
X = train_data.drop(columns=target_cols)
y = train_data[target_cols]

# Handle missing targets
total_nan = y.isnull().any(axis=1).sum()
if total_nan > 0:
    print(f"\n⚠ Removing {total_nan} rows with missing target values")
    valid_idx = ~y.isnull().any(axis=1)
    X = X[valid_idx].reset_index(drop=True)
    y = y[valid_idx].reset_index(drop=True)

# Convert to Series for single target
if not is_multi_target:
    y = y.iloc[:, 0]

# Detect feature types
cat_cols, num_cols = detect_feature_types(X)

print(f"\n✓ Feature summary:")
print(f"  Total features: {X.shape[1]}")
print(f"  Categorical: {len(cat_cols)}")
print(f"  Numerical: {len(num_cols)}")

if cat_cols:
    print(f"\n  Categorical columns: {cat_cols}")
if num_cols:
    print(f"\n  Numerical columns: {num_cols}")


# ============================================================================
# 7. EXPLORATORY DATA ANALYSIS (Optional)
# ============================================================================

if CONFIG['do_plotting']:
    print_section("EXPLORATORY DATA ANALYSIS")

    # Numerical features - distributions
    if len(num_cols) > 0:
        print("\n✓ Plotting numerical feature distributions...")
        n_num = min(len(num_cols), 12)  # Limit to 12 plots
        ncols = 3
        nrows = (n_num + ncols - 1) // ncols

        fig, axes = plt.subplots(nrows, ncols, figsize=(15, 4*nrows))
        axes = axes.flatten() if n_num > 1 else [axes]

        for i, col in enumerate(num_cols[:n_num]):
            sns.histplot(X[col].dropna(), kde=True, ax=axes[i])
            axes[i].set_title(col, fontsize=10)

        for i in range(n_num, len(axes)):
            axes[i].axis('off')

        plt.tight_layout()
        plt.show()

    # Categorical features - top categories
    if len(cat_cols) > 0:
        print("\n✓ Plotting categorical feature distributions...")
        for col in cat_cols[:5]:  # Limit to 5
            plt.figure(figsize=(10, 4))
            top_cats = X[col].value_counts().head(15)
            sns.barplot(x=top_cats.values, y=top_cats.index)
            plt.title(f"Top Categories: {col}")
            plt.xlabel("Count")
            plt.tight_layout()
            plt.show()


# ============================================================================
# 8. PREPROCESSING PIPELINE
# ============================================================================

print_section("DATA PREPROCESSING")

# Imputation
print("\n✓ Setting up imputation...")
if num_cols:
    num_imputer = SimpleImputer(strategy='median')
    X[num_cols] = num_imputer.fit_transform(X[num_cols])
    test_data[num_cols] = num_imputer.transform(test_data[num_cols])
    print("  - Numerical: median imputation")

if cat_cols:
    cat_imputer = SimpleImputer(strategy='most_frequent')
    X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])
    test_data[cat_cols] = cat_imputer.transform(test_data[cat_cols])
    print("  - Categorical: most frequent imputation")

# Outlier capping (optional)
if CONFIG['do_outlier_cap'] and num_cols:
    print("\n✓ Capping outliers (IQR method)...")
    for col in num_cols:
        q1 = X[col].quantile(0.25)
        q3 = X[col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr

        X[col] = X[col].clip(lower=lower, upper=upper)
        test_data[col] = test_data[col].clip(lower=lower, upper=upper)

# Create preprocessing pipeline
print("\n✓ Building preprocessing pipeline...")
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
        ('num', StandardScaler(), num_cols)
    ],
    remainder='passthrough'
)


# ============================================================================
# 6. TRAIN/VALIDATION SPLIT & TRANSFORMATION
# ============================================================================

print_section("TRAIN/VALIDATION SPLIT")

stratify_param = y if not is_multi_target else None

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=CONFIG['validation_size'],
    random_state=RANDOM_STATE,
    stratify=stratify_param
)

print(f"\n✓ Train: {X_train.shape[0]}, Val: {X_val.shape[0]}")

# Transform features
print("✓ Transforming features...")
X_train_pre = preprocessor.fit_transform(X_train)
X_val_pre = preprocessor.transform(X_val)
test_data_pre = preprocessor.transform(test_data)

print(f"  Shape: {X_train_pre.shape[1]} features after encoding")


# ============================================================================
# 7. LABEL ENCODING
# ============================================================================

print_section("ENCODING TARGETS")

if not is_multi_target:
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_val_enc = le.transform(y_val)
    print(f"\n✓ Encoded {len(le.classes_)} classes")
else:
    label_encoders = {}
    y_train_enc = pd.DataFrame(index=y_train.index)
    y_val_enc = pd.DataFrame(index=y_val.index)

    for target_col in target_cols:
        le_temp = LabelEncoder()
        y_train_enc[target_col] = le_temp.fit_transform(y_train[target_col])
        y_val_enc[target_col] = le_temp.transform(y_val[target_col])
        label_encoders[target_col] = le_temp

    y_train_enc = y_train_enc.values
    y_val_enc = y_val_enc.values
    print(f"\n✓ Encoded {len(target_cols)} targets")


# ============================================================================
# 8. MODEL TRAINING
# ============================================================================

print_section("MODEL TRAINING")

# Select and create model
if CONFIG['model_type'] == 'fast_ensemble':
    base_model = get_fast_ensemble(
        ensemble_models=CONFIG.get('fast_ensemble_models', ['lightgbm', 'xgboost']),
        n_estimators=CONFIG['n_estimators'],
        use_class_weight=CONFIG['use_class_weight'],
        random_state=RANDOM_STATE
    )
else:
    print(f"\n✓ Model: {CONFIG['model_type'].upper()}")
    base_model = get_model(
        model_type=CONFIG['model_type'],
        n_estimators=CONFIG['n_estimators'],
        use_class_weight=CONFIG['use_class_weight'],
        random_state=RANDOM_STATE
    )

# Wrap for multi-target if needed
if is_multi_target:
    model = MultiOutputClassifier(base_model, n_jobs=-1)
else:
    model = base_model

# Train
print("\n⏳ Training...")
train_start = time.time()
model.fit(X_train_pre, y_train_enc)
train_time = time.time() - train_start
print(f"✓ Training completed in {train_time:.1f}s")


# ============================================================================
# 9. MODEL EVALUATION
# ============================================================================

print_section("EVALUATION")

# Make predictions
y_val_pred = model.predict(X_val_pre)

if is_multi_target:
    print("\n📊 Multi-Target Metrics:")
    for idx, target_col in enumerate(target_cols):
        acc = accuracy_score(y_val_enc[:, idx], y_val_pred[:, idx])
        print(f"  {target_col}: Accuracy={acc:.4f}")
else:
    # Get probabilities
    try:
        y_val_proba = model.predict_proba(X_val_pre)

        # Calculate metrics
        loss_val = log_loss(y_val_enc, y_val_proba)
        acc_val = accuracy_score(y_val_enc, y_val_pred)

        print(f"\n📊 Validation Metrics:")
        print(f"  Log Loss: {loss_val:.4f}")
        print(f"  Accuracy: {acc_val:.4f}")

    except Exception as e:
        print(f"\n⚠ Error: {e}")


# ============================================================================
# 13. HYPERPARAMETER TUNING (Optional)
# ============================================================================

tuned_model = None
if CONFIG['do_hyperparam_tuning']:
    print_section("HYPERPARAMETER TUNING")

    print("\n⏳ Starting RandomizedSearchCV...")

    # Define parameter distributions based on model type
    model_type_for_tuning = CONFIG['model_type']
    if model_type_for_tuning in ['voting_ensemble', 'stacking_ensemble']:
        print("  ℹ Skipping hyperparameter tuning for ensemble models (using base parameters)")
        tuned_model = None
    else:
        param_prefix = "estimator__" if is_multi_target else ""

        if model_type_for_tuning == 'xgboost' and XGBOOST_AVAILABLE:
            param_dist = {
                f"{param_prefix}n_estimators": [500, 800, 1000, 1500],
                f"{param_prefix}max_depth": [4, 6, 8, 10],
                f"{param_prefix}learning_rate": [0.01, 0.05, 0.1, 0.2],
                f"{param_prefix}min_child_weight": [1, 3, 5],
                f"{param_prefix}subsample": [0.6, 0.8, 1.0],
                f"{param_prefix}colsample_bytree": [0.6, 0.8, 1.0],
            }
        elif model_type_for_tuning == 'lightgbm' and LIGHTGBM_AVAILABLE:
            param_dist = {
                f"{param_prefix}n_estimators": [500, 800, 1000, 1500],
                f"{param_prefix}max_depth": [4, 6, 8, 10],
                f"{param_prefix}learning_rate": [0.01, 0.05, 0.1, 0.2],
                f"{param_prefix}num_leaves": [31, 50, 70, 100],
                f"{param_prefix}subsample": [0.6, 0.8, 1.0],
                f"{param_prefix}colsample_bytree": [0.6, 0.8, 1.0],
            }
        elif model_type_for_tuning == 'catboost' and CATBOOST_AVAILABLE:
            param_dist = {
                f"{param_prefix}iterations": [500, 800, 1000, 1500],
                f"{param_prefix}depth": [4, 6, 8, 10],
                f"{param_prefix}learning_rate": [0.01, 0.05, 0.1, 0.2],
            }
        elif model_type_for_tuning == 'random_forest':
            param_dist = {
                f"{param_prefix}n_estimators": [200, 500, 800, 1000],
                f"{param_prefix}max_depth": [None, 10, 15, 20, 25],
                f"{param_prefix}min_samples_split": [2, 5, 10],
                f"{param_prefix}min_samples_leaf": [1, 2, 4],
                f"{param_prefix}max_features": ["sqrt", "log2", 0.3]
            }
        elif model_type_for_tuning == 'gradient_boosting':
            param_dist = {
                f"{param_prefix}n_estimators": [100, 200, 300, 500],
                f"{param_prefix}max_depth": [3, 5, 7, 9],
                f"{param_prefix}learning_rate": [0.01, 0.05, 0.1, 0.2],
                f"{param_prefix}subsample": [0.6, 0.8, 1.0],
            }
        elif model_type_for_tuning == 'extra_trees':
            param_dist = {
                f"{param_prefix}n_estimators": [200, 500, 800, 1000],
                f"{param_prefix}max_depth": [None, 10, 15, 20, 25],
                f"{param_prefix}min_samples_split": [2, 5, 10],
                f"{param_prefix}min_samples_leaf": [1, 2, 4],
            }
        else:
            print(f"  ℹ No hyperparameter tuning defined for {model_type_for_tuning}")
            param_dist = None

        if param_dist:
            # Choose scoring
            if is_multi_target:
                scoring = 'accuracy'
            else:
                n_classes = len(le.classes_)
                scoring = 'roc_auc_ovr' if n_classes > 2 else 'roc_auc'

            # Run search
            rnd_search = RandomizedSearchCV(
                estimator=model,
                param_distributions=param_dist,
                n_iter=CONFIG['n_iter_search'],
                scoring=scoring,
                cv=3,
                verbose=1,
                random_state=RANDOM_STATE,
                n_jobs=-1
            )

            rnd_search.fit(X_train_pre, y_train_enc)

            print("\n✓ Best parameters:")
            for param, value in rnd_search.best_params_.items():
                print(f"  {param}: {value}")
            print(f"\nBest CV score: {rnd_search.best_score_:.4f}")

            tuned_model = rnd_search.best_estimator_

            # Evaluate tuned model
            y_val_pred_tuned = tuned_model.predict(X_val_pre)

            if not is_multi_target:
                try:
                    y_val_proba_tuned = tuned_model.predict_proba(X_val_pre)
                    loss_val_tuned = log_loss(y_val_enc, y_val_proba_tuned)
                    acc_val_tuned = accuracy_score(y_val_enc, y_val_pred_tuned)

                    print(f"\n📊 Tuned Model - Validation Metrics:")
                    print(f"  Log Loss: {loss_val_tuned:.4f}")
                    print(f"  Accuracy: {acc_val_tuned:.4f}")

                    if loss_val_tuned < loss_val:
                        print("  ✓ Improvement over baseline!")
                except:
                    pass


# ============================================================================
# 14. GENERATE PREDICTIONS
# ============================================================================

print_section("GENERATING PREDICTIONS")

final_model = tuned_model if tuned_model is not None else model
print(f"\n✓ Using {'TUNED' if tuned_model else 'BASELINE'} model")

# Determine submission format
if CONFIG['submission_type'] == 'probabilities':
    print("✓ Generating probability predictions (for Log Loss)")

    if is_multi_target:
        # Multi-target probabilities not commonly used, generate classes
        print("  ℹ Multi-target with probabilities not standard, using class predictions")
        test_pred = final_model.predict(test_data_pre)

        submission_df = pd.DataFrame({CONFIG['id_col']: test_ids})
        for idx, target_col in enumerate(target_cols):
            le_temp = label_encoders[target_col]
            submission_df[target_col] = le_temp.inverse_transform(test_pred[:, idx])
    else:
        # Generate probabilities for each class
        test_proba = final_model.predict_proba(test_data_pre)

        # Create submission DataFrame
        if test_ids is not None:
            submission_df = pd.DataFrame({CONFIG['id_col']: test_ids})
        else:
            submission_df = pd.DataFrame()

        # Determine column order from sample submission if available
        if sample_submission is not None:
            # Get probability column names from sample submission (exclude ID column)
            expected_columns = [col for col in sample_submission.columns if col != CONFIG['id_col']]

            print(f"\n  Using column order from sample submission:")
            print(f"  Expected columns: {expected_columns}")

            # Use the exact columns from sample submission
            class_to_idx = {class_name: idx for idx, class_name in enumerate(le.classes_)}

            # Create variations to handle prefixes
            class_name_variations = {}
            for class_name in le.classes_:
                class_name_variations[class_name] = class_name
                for prefix in ['Status_', 'status_', 'TARGET_', 'target_', 'Class_', 'class_']:
                    if class_name.startswith(prefix):
                        clean_name = class_name[len(prefix):]
                        class_name_variations[clean_name] = class_name
                    prefixed = prefix + class_name
                    class_name_variations[prefixed] = class_name

            # Add columns in exact order from sample submission
            columns_added = 0
            for col_name in expected_columns:
                matched_class = None

                if col_name in class_to_idx:
                    matched_class = col_name
                elif col_name in class_name_variations:
                    matched_class = class_name_variations[col_name]
                else:
                    for prefix in ['Status_', 'status_', 'TARGET_', 'target_', 'Class_', 'class_']:
                        if col_name.startswith(prefix):
                            potential_match = col_name[len(prefix):]
                            if potential_match in class_to_idx:
                                matched_class = potential_match
                                break

                    if matched_class is None:
                        for class_name in le.classes_:
                            if col_name.endswith(class_name):
                                matched_class = class_name
                                break

                if matched_class and matched_class in class_to_idx:
                    idx = class_to_idx[matched_class]
                    submission_df[col_name] = test_proba[:, idx]
                    columns_added += 1
                else:
                    print(f"  ⚠ Warning: Column '{col_name}' could not be mapped")
                    submission_df[col_name] = 0.0

            print(f"  ✓ Mapped {columns_added}/{len(expected_columns)} columns from sample submission")

        else:
            # No sample submission - use prefix from config
            print(f"\n  No sample submission - using config settings")

            prefix = CONFIG.get('submission_column_prefix', '')
            if prefix:
                print(f"  Adding prefix '{prefix}' to column names")

            # Sort class names to ensure proper order (Q1, Q2, ..., Q10)
            classes_sorted = sorted(le.classes_, key=lambda x: (
                int(''.join(filter(str.isdigit, str(x)))) if any(c.isdigit() for c in str(x)) else 0,
                str(x)
            ))

            # Create mapping from sorted classes to probability indices
            class_to_idx = {class_name: idx for idx, class_name in enumerate(le.classes_)}

            for class_name in classes_sorted:
                idx = class_to_idx[class_name]
                # Add prefix if specified
                column_name = f"{prefix}{class_name}" if prefix else class_name
                submission_df[column_name] = test_proba[:, idx]

        print(f"\n  Generated probabilities for {len(le.classes_)} classes")
        print(f"  Final column order: {list(submission_df.columns)}")

else:  # 'classes'
    print("✓ Generating class predictions")

    test_pred = final_model.predict(test_data_pre)

    if is_multi_target:
        submission_df = pd.DataFrame({CONFIG['id_col']: test_ids})
        for idx, target_col in enumerate(target_cols):
            le_temp = label_encoders[target_col]
            submission_df[target_col] = le_temp.inverse_transform(test_pred[:, idx])
    else:
        test_pred_decoded = le.inverse_transform(test_pred)

        if test_ids is not None:
            submission_df = pd.DataFrame({
                CONFIG['id_col']: test_ids,
                target_cols[0]: test_pred_decoded
            })
        else:
            submission_df = pd.DataFrame({target_cols[0]: test_pred_decoded})

# Save submission
submission_df.to_csv(CONFIG['output_file'], index=False)
print(f"\n✓ Submission saved to '{CONFIG['output_file']}'")

# Verify format matches sample submission
if sample_submission is not None:
    print("\n" + "-"*80)
    print("FORMAT VERIFICATION")
    print("-"*80)

    # Check column match
    expected_cols = list(sample_submission.columns)
    actual_cols = list(submission_df.columns)

    if expected_cols == actual_cols:
        print("✅ Column names and order MATCH sample submission perfectly!")
    else:
        print("⚠ Column differences detected:")
        print(f"  Expected: {expected_cols}")
        print(f"  Actual:   {actual_cols}")

        missing = set(expected_cols) - set(actual_cols)
        extra = set(actual_cols) - set(expected_cols)
        if missing:
            print(f"  Missing columns: {missing}")
        if extra:
            print(f"  Extra columns: {extra}")

    # Check shape
    if len(submission_df) == len(sample_submission):
        print(f"✅ Row count matches: {len(submission_df)} rows")
    else:
        print(f"⚠ Row count mismatch:")
        print(f"  Expected: {len(sample_submission)} rows")
        print(f"  Actual:   {len(submission_df)} rows")

# Display statistics
print("\n" + "-"*80)
print("SUBMISSION SUMMARY")
print("-"*80)
print(f"Total rows: {len(submission_df)}")
print(f"Columns: {list(submission_df.columns)}")

print("\nFirst 10 rows:")
print(submission_df.head(10))

if CONFIG['submission_type'] == 'probabilities' and not is_multi_target:
    # Verify probabilities sum to 1
    prob_cols = [c for c in submission_df.columns if c != CONFIG['id_col']]
    prob_sum = submission_df[prob_cols].sum(axis=1)
    print(f"\nProbability verification:")
    print(f"  Min sum: {prob_sum.min():.6f}")
    print(f"  Max sum: {prob_sum.max():.6f}")
    print(f"  All sums ≈ 1.0: {np.allclose(prob_sum, 1.0)}")

print("\n" + "="*80)
print("✅ PIPELINE COMPLETED SUCCESSFULLY!")
print("="*80)
print(f"\n💡 Model Performance Tips:")
print(f"  - Current model: {CONFIG['model_type']}")
if CONFIG['model_type'] not in ['voting_ensemble', 'stacking_ensemble']:
    print(f"  - Try ensemble methods for better accuracy:")
    print(f"    • 'voting_ensemble': Combines multiple models by voting")
    print(f"    • 'stacking_ensemble': Uses meta-model to combine predictions")
print(f"  - Best models for most datasets:")
print(f"    1. XGBoost (fast, accurate, handles missing values)")
print(f"    2. LightGBM (very fast, great for large datasets)")
print(f"    3. CatBoost (excellent for categorical features)")
print(f"    4. Stacking Ensemble (highest accuracy, slower)")
print(f"\n💡 Configuration Tips:")
print(f"  - For Log Loss competitions: submission_type='probabilities'")
print(f"  - Enable hyperparameter tuning: do_hyperparam_tuning=True")
print(f"  - Use sample_submission.csv for exact format matching")
print(f"  - Increase n_estimators (1000-2000) for better accuracy")